In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [9]:
from googleapiclient.discovery import build

def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [10]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1MH-ufKNr2U_OY-L9p5rknxinAb225jbcp55irHUCdoE'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID, 'b_block1')
b1 = pd.DataFrame(data[1:], columns=data[0])

data = pull_sheet_data(SCOPES,SPREADSHEET_ID, 'b_block2')
b2 = pd.DataFrame(data[1:] )
b2 = b2[b2.columns[0:5]]
b2.columns = data[0]

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=392396298323-4cbputrpiaqen1tm0muv3dv1is72ejhh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53632%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=tpicaoE69Nv4HpIGGvoNwYMI0QugX3&access_type=offline
COMPLETE: Data copied
COMPLETE: Data copied


In [11]:
# b1 = pd.read_excel('Raspi_data.xlsx' , sheet_name= 'b_block1')
# b2 = pd.read_excel('Raspi_data.xlsx' , sheet_name = 'b_block2')

In [12]:
b1.drop('cputemp' ,axis= 1 , inplace = True)
b2.drop('cputemp' , axis = 1 ,inplace = True)

In [13]:
b1.head()

,Datetime,Vplus,Qv,Vminus
0,2022-03-08 16:00,348.5037842,0.008165987208,-0.2556762695
1,2022-03-08 18:00,348.5195007,0.007397520822,-0.2556762695
2,2022-03-08 22:00,348.5542908,0.008760321885,-0.2557067871
3,2022-03-09 00:00,348.5714722,0.007152980193,-0.2556762695
4,2022-03-09 02:00,348.5881653,0.008557212539,-0.2557373047


In [14]:
# b2 = b2[['time', 'Vplus', 'Qv', 'Vminus']]
b2.head()

,time,Vplus,Qv,Vminus
0,2022-03-08 18:30,475.6612854,0.003352362197,-92.29959106
1,2022-03-08 20:30,475.6637268,0.004518549424,-92.30050659
2,2022-03-08 22:30,475.6658936,0.004370904993,-92.30154419
3,2022-03-09 00:30,475.6676025,-0.003289248329,-92.30291748
4,2022-03-09 02:30,475.6690063,-0.00124421122,-92.30465698


In [15]:
b1.isnull().sum()

Datetime    0
Vplus       0
Qv          0
Vminus      0
dtype: int64

In [16]:
b1[[ 'Vplus', 'Qv', 'Vminus']] = b1[[ 'Vplus', 'Qv', 'Vminus']].astype(float)
b2[[ 'Vplus', 'Qv', 'Vminus']] = b2[[ 'Vplus', 'Qv', 'Vminus']].astype(float)

In [17]:
b1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  2051 non-null   object 
 1   Vplus     2051 non-null   float64
 2   Qv        2051 non-null   float64
 3   Vminus    2051 non-null   float64
dtypes: float64(3), object(1)
memory usage: 64.2+ KB


In [18]:
b2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624 entries, 0 to 2623
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    2624 non-null   object 
 1   Vplus   2624 non-null   float64
 2   Qv      2624 non-null   float64
 3   Vminus  2624 non-null   float64
dtypes: float64(3), object(1)
memory usage: 82.1+ KB


In [19]:
b1['Datetime'] = pd.to_datetime( b1['Datetime'])
b2['time'] = pd.to_datetime( b2['time'])
b2['Datetime'] = b2['time']

In [20]:
b1['weekday'] = b1['Datetime'].apply(lambda datetime : datetime.isocalendar().weekday)
b1['month'] = b1['Datetime'].apply(lambda datetime : datetime.month)
b1['year'] = b1['Datetime'].apply(lambda datetime : datetime.year)
b1['day'] = b1['Datetime'].apply(lambda datetime : datetime.day)
b1['time'] = b1['Datetime'].apply(lambda datetime : datetime.time())
b1['Date'] = b1['Datetime'].dt.date

In [21]:
b2['weekday'] = b2['time'].apply(lambda datetime : datetime.isocalendar().weekday)
b2['month'] = b2['time'].apply(lambda datetime : datetime.month)
b2['year'] = b2['time'].apply(lambda datetime : datetime.year)
b2['day'] = b2['time'].apply(lambda datetime : datetime.day)
b2['time'] = b2['time'].apply(lambda datetime : datetime.time())
b2['Date'] = b2['Datetime'].dt.date

In [22]:
b1.head()

,Datetime,Vplus,Qv,Vminus,weekday,month,year,day,time,Date
0,2022-03-08 16:00:00,348.503784,0.008166,-0.255676,2,3,2022,8,16:00:00,2022-03-08
1,2022-03-08 18:00:00,348.519501,0.007398,-0.255676,2,3,2022,8,18:00:00,2022-03-08
2,2022-03-08 22:00:00,348.554291,0.008760,-0.255707,2,3,2022,8,22:00:00,2022-03-08
3,2022-03-09 00:00:00,348.571472,0.007153,-0.255676,3,3,2022,9,00:00:00,2022-03-09
4,2022-03-09 02:00:00,348.588165,0.008557,-0.255737,3,3,2022,9,02:00:00,2022-03-09


In [23]:
b2.head()

,time,Vplus,Qv,Vminus,Datetime,weekday,month,year,day,Date
0,18:30:00,475.661285,0.003352,-92.299591,2022-03-08 18:30:00,2,3,2022,8,2022-03-08
1,20:30:00,475.663727,0.004519,-92.300507,2022-03-08 20:30:00,2,3,2022,8,2022-03-08
2,22:30:00,475.665894,0.004371,-92.301544,2022-03-08 22:30:00,2,3,2022,8,2022-03-08
3,00:30:00,475.667602,-0.003289,-92.302917,2022-03-09 00:30:00,3,3,2022,9,2022-03-09
4,02:30:00,475.669006,-0.001244,-92.304657,2022-03-09 02:30:00,3,3,2022,9,2022-03-09


In [24]:
b1.drop(['Qv' , 'Vminus' , 'Datetime'] , axis = 1 , inplace = True )
b2.drop(['Qv' , 'Vminus' , 'Datetime'] , axis = 1 , inplace = True )

In [25]:
b1.head()

,Vplus,weekday,month,year,day,time,Date
0,348.503784,2,3,2022,8,16:00:00,2022-03-08
1,348.519501,2,3,2022,8,18:00:00,2022-03-08
2,348.554291,2,3,2022,8,22:00:00,2022-03-08
3,348.571472,3,3,2022,9,00:00:00,2022-03-09
4,348.588165,3,3,2022,9,02:00:00,2022-03-09


In [26]:
b2.head()

,time,Vplus,weekday,month,year,day,Date
0,18:30:00,475.661285,2,3,2022,8,2022-03-08
1,20:30:00,475.663727,2,3,2022,8,2022-03-08
2,22:30:00,475.665894,2,3,2022,8,2022-03-08
3,00:30:00,475.667602,3,3,2022,9,2022-03-09
4,02:30:00,475.669006,3,3,2022,9,2022-03-09


In [27]:
max_b1 = b1.groupby(['year','month' ,'day'] , group_keys = True).max().reset_index()
max_b1.set_index('Date' , inplace = True)
max_b1

,year,month,day,Vplus,weekday,time
Date,,,,,,
2022-03-08,2022,3,8,348.554291,2,22:00:00
2022-03-09,2022,3,9,348.744110,3,22:00:00
2022-03-10,2022,3,10,348.848114,4,12:00:00
2022-03-14,2022,3,14,356.915619,1,18:00:00
2022-03-15,2022,3,15,368.940155,2,16:38:00
...,...,...,...,...,...,...
2023-04-13,2023,4,13,20094.806640,4,18:00:00
2023-04-14,2023,4,14,20144.042970,5,22:00:00
2023-04-15,2023,4,15,20199.726560,6,22:00:00


In [28]:
min_b1 = b1.groupby(['year','month' ,'day'] , group_keys = True).min().reset_index()
min_b1.set_index('Date' , inplace = True)
min_b1

,year,month,day,Vplus,weekday,time
Date,,,,,,
2022-03-08,2022,3,8,348.503784,2,16:00:00
2022-03-09,2022,3,9,348.571472,3,00:00:00
2022-03-10,2022,3,10,348.759399,4,00:00:00
2022-03-14,2022,3,14,356.915619,1,18:00:00
2022-03-15,2022,3,15,368.734619,2,16:36:00
...,...,...,...,...,...,...
2023-04-13,2023,4,13,20066.220700,4,00:00:00
2023-04-14,2023,4,14,20107.583980,5,02:00:00
2023-04-15,2023,4,15,20146.128910,6,00:00:00


In [29]:
max_b2 = b2.groupby(['year','month' ,'day'] , group_keys = True).max().reset_index()
max_b2.set_index('Date' , inplace = True)
max_b2

,year,month,day,time,Vplus,weekday
Date,,,,,,
2022-03-08,2022,3,8,22:30:00,475.665894,2
2022-03-09,2022,3,9,22:30:00,475.698120,3
2022-03-10,2022,3,10,12:30:00,475.722107,4
2022-03-14,2022,3,14,18:30:00,490.450409,1
2022-03-16,2022,3,16,22:30:00,510.588593,3
...,...,...,...,...,...,...
2023-04-13,2023,4,13,22:30:00,12726.666990,4
2023-04-14,2023,4,14,22:30:00,12767.888670,5
2023-04-15,2023,4,15,22:30:00,12810.455080,6


In [30]:
min_b2 = b2.groupby(['year','month' ,'day'] , group_keys = True).min().reset_index()
min_b2.set_index('Date' , inplace = True)
min_b2

,year,month,day,time,Vplus,weekday
Date,,,,,,
2022-03-08,2022,3,8,18:30:00,475.661285,2
2022-03-09,2022,3,9,00:30:00,475.667602,3
2022-03-10,2022,3,10,00:30:00,475.700440,4
2022-03-14,2022,3,14,18:30:00,490.450409,1
2022-03-16,2022,3,16,18:30:00,510.570618,3
...,...,...,...,...,...,...
2023-04-13,2023,4,13,00:30:00,12692.512700,4
2023-04-14,2023,4,14,00:30:00,12731.066410,5
2023-04-15,2023,4,15,00:30:00,12772.380860,6


In [31]:
boys_hostel = min_b1.copy()[['year','month' ,'day' , 'weekday']]
boys_hostel['b1_usage'] = max_b1['Vplus']-min_b1['Vplus']
boys_hostel['b2_usage'] = max_b2['Vplus']-min_b2['Vplus']
# for missing data records
boys_hostel.fillna(boys_hostel.mean() , inplace = True)

In [32]:
boys_hostel['total_usage'] = boys_hostel['b1_usage'] + boys_hostel['b2_usage']

In [33]:
boys_hostel

,year,month,day,weekday,b1_usage,b2_usage,total_usage
Date,,,,,,,
2022-03-08,2022,3,8,2,0.050507,0.004608,0.055115
2022-03-09,2022,3,9,3,0.172638,0.030518,0.203155
2022-03-10,2022,3,10,4,0.088715,0.021667,0.110382
2022-03-14,2022,3,14,1,0.000000,0.000000,0.000000
2022-03-15,2022,3,15,2,0.205536,24.998359,25.203894
...,...,...,...,...,...,...,...
2023-04-13,2023,4,13,4,28.585940,34.154290,62.740230
2023-04-14,2023,4,14,5,36.458990,36.822260,73.281250
2023-04-15,2023,4,15,6,53.597650,38.074220,91.671870


In [34]:
from sklearn.model_selection import train_test_split

In [35]:
b1_x_train, b1_x_test, b1_y_train, b1_y_test = train_test_split(boys_hostel[['day','weekday' , 'month']], boys_hostel['b1_usage'], test_size=0.33, random_state=1, shuffle=False)
b2_x_train, b2_x_test, b2_y_train, b2_y_test = train_test_split(boys_hostel[['day','weekday' , 'month']], boys_hostel['b2_usage'], test_size=0.33, random_state=1, shuffle=False)

In [36]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

In [37]:
DM_train = xgb.DMatrix(data=b1_x_train, label=b1_y_train)
grid_param = {"learning_rate": [0.01, 0.001, 0.1],
              "n_estimators": [100, 150, 200 , 250 , 300 , 350, 400],
              "alpha": [0.1,0.75 , 0.5, 1],
              "max_depth": [2, 3, 4 , 6, 9 , 11]}
model = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=model, param_grid=grid_param,
                       scoring="neg_mean_squared_error",
                       cv=4, verbose=1)
grid_mse.fit(b1_x_train, b1_y_train)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 504 candidates, totalling 2016 fits
Best parameters found:  {'alpha': 1, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 200}
Lowest RMSE found:  20.822445260716826


In [40]:
"""
Fitting 4 folds for each of 504 candidates, totalling 2016 fits
Best parameters found:  {'alpha': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}
Lowest RMSE found:  23.223646069996096
"""
b1_xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, **grid_mse.best_params_)
b1_xgb_model.fit(b1_x_train, b1_y_train)
b1_xgb_prediction = b1_xgb_model.predict(b1_x_test)
mse(b1_y_test , b1_xgb_prediction)

452.02581284636557

In [41]:
DM_train = xgb.DMatrix(data=b2_x_train, label=b2_y_train)
grid_param = {"learning_rate": [0.01, 0.001, 0.1],
              "n_estimators": [100, 150, 200 , 250 , 300 , 350, 400],
              "alpha": [0.1,0.75 , 0.5, 1],
              "max_depth": [2, 3, 4 , 6, 9 , 11]}
model = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=model, param_grid=grid_param,
                       scoring="neg_mean_squared_error",
                       cv=4, verbose=1)
grid_mse.fit(b2_x_train, b2_y_train)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 504 candidates, totalling 2016 fits
Best parameters found:  {'alpha': 0.1, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 300}
Lowest RMSE found:  8.504806890775242


In [42]:
b2_xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, **grid_mse.best_params_)
b2_xgb_model.fit(b2_x_train, b2_y_train)
b2_xgb_prediction = b2_xgb_model.predict(b2_x_test)
mse(b2_y_test , b2_xgb_prediction)

103.21234506115727

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000 ], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001 ,'scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
              } 
  
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END .C=0.1, gamma=1, kernel=linear;, score=-1868.760 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=1, kernel=linear;, score=-337.915 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=1, kernel=linear;, score=-381.609 total time=   0.0s
[CV 4/5] END ..C=0.1, gamma=1, kernel=linear;, score=-505.693 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=1, kernel=linear;, score=-330.434 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=1, kernel=poly;, score=-1890.781 total time=  26.7s
[CV 2/5] END ....C=0.1, gamma=1, kernel=poly;, score=-490.859 total time=  22.4s
[CV 3/5] END ....C=0.1, gamma=1, kernel=poly;, score=-361.243 total time=  17.3s
[CV 4/5] END ....C=0.1, gamma=1, kernel=poly;, score=-538.965 total time= 1.6min
[CV 5/5] END ....C=0.1, gamma=1, kernel=poly;, score=-653.052 total time=   7.1s
[CV 1/5] END ....C=0.1, gamma=1, kernel=rbf;, score=-1609.757 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=rbf;,

[CV 1/5] END C=0.1, gamma=1e-05, kernel=rbf;, score=-1610.301 total time=   0.0s
[CV 2/5] END .C=0.1, gamma=1e-05, kernel=rbf;, score=-337.715 total time=   0.0s
[CV 3/5] END .C=0.1, gamma=1e-05, kernel=rbf;, score=-371.464 total time=   0.0s
[CV 4/5] END .C=0.1, gamma=1e-05, kernel=rbf;, score=-439.416 total time=   0.0s
[CV 5/5] END .C=0.1, gamma=1e-05, kernel=rbf;, score=-265.828 total time=   0.0s
[CV 1/5] END C=0.1, gamma=1e-05, kernel=sigmoid;, score=-1610.333 total time=   0.0s
[CV 2/5] END C=0.1, gamma=1e-05, kernel=sigmoid;, score=-337.711 total time=   0.0s
[CV 3/5] END C=0.1, gamma=1e-05, kernel=sigmoid;, score=-371.465 total time=   0.0s
[CV 4/5] END C=0.1, gamma=1e-05, kernel=sigmoid;, score=-439.401 total time=   0.0s
[CV 5/5] END C=0.1, gamma=1e-05, kernel=sigmoid;, score=-265.823 total time=   0.0s
[CV 1/5] END C=0.1, gamma=scale, kernel=linear;, score=-1868.760 total time=   0.0s
[CV 2/5] END C=0.1, gamma=scale, kernel=linear;, score=-337.915 total time=   0.0s
[CV 3/5

[CV 5/5] END C=1, gamma=0.0001, kernel=linear;, score=-484.176 total time=   0.0s
[CV 1/5] END C=1, gamma=0.0001, kernel=poly;, score=-1610.345 total time=   0.0s
[CV 2/5] END .C=1, gamma=0.0001, kernel=poly;, score=-337.705 total time=   0.0s
[CV 3/5] END .C=1, gamma=0.0001, kernel=poly;, score=-371.468 total time=   0.0s
[CV 4/5] END .C=1, gamma=0.0001, kernel=poly;, score=-439.394 total time=   0.0s
[CV 5/5] END .C=1, gamma=0.0001, kernel=poly;, score=-265.823 total time=   0.0s
[CV 1/5] END .C=1, gamma=0.0001, kernel=rbf;, score=-1604.013 total time=   0.0s
[CV 2/5] END ..C=1, gamma=0.0001, kernel=rbf;, score=-338.365 total time=   0.0s
[CV 3/5] END ..C=1, gamma=0.0001, kernel=rbf;, score=-371.276 total time=   0.0s
[CV 4/5] END ..C=1, gamma=0.0001, kernel=rbf;, score=-441.878 total time=   0.0s
[CV 5/5] END ..C=1, gamma=0.0001, kernel=rbf;, score=-266.626 total time=   0.0s
[CV 1/5] END C=1, gamma=0.0001, kernel=sigmoid;, score=-1607.134 total time=   0.0s
[CV 2/5] END C=1, gamma=

[CV 1/5] END ..C=10, gamma=0.1, kernel=poly;, score=-1940.314 total time=   8.3s
[CV 2/5] END ...C=10, gamma=0.1, kernel=poly;, score=-386.161 total time=   2.4s
[CV 3/5] END ...C=10, gamma=0.1, kernel=poly;, score=-360.530 total time=   6.2s
[CV 4/5] END ...C=10, gamma=0.1, kernel=poly;, score=-534.680 total time=  18.8s
[CV 5/5] END ...C=10, gamma=0.1, kernel=poly;, score=-579.663 total time=   5.9s
[CV 1/5] END ...C=10, gamma=0.1, kernel=rbf;, score=-1663.303 total time=   0.0s
[CV 2/5] END ....C=10, gamma=0.1, kernel=rbf;, score=-324.106 total time=   0.0s
[CV 3/5] END ....C=10, gamma=0.1, kernel=rbf;, score=-472.881 total time=   0.0s
[CV 4/5] END ....C=10, gamma=0.1, kernel=rbf;, score=-464.325 total time=   0.0s
[CV 5/5] END ....C=10, gamma=0.1, kernel=rbf;, score=-355.053 total time=   0.0s
[CV 1/5] END C=10, gamma=0.1, kernel=sigmoid;, score=-1610.095 total time=   0.0s
[CV 2/5] END C=10, gamma=0.1, kernel=sigmoid;, score=-337.466 total time=   0.0s
[CV 3/5] END C=10, gamma=0.

[CV 2/5] END ..C=10, gamma=scale, kernel=rbf;, score=-376.111 total time=   0.0s
[CV 3/5] END ..C=10, gamma=scale, kernel=rbf;, score=-425.621 total time=   0.0s
[CV 4/5] END ..C=10, gamma=scale, kernel=rbf;, score=-485.504 total time=   0.0s
[CV 5/5] END ..C=10, gamma=scale, kernel=rbf;, score=-370.644 total time=   0.0s
[CV 1/5] END C=10, gamma=scale, kernel=sigmoid;, score=-2170.384 total time=   0.0s
[CV 2/5] END C=10, gamma=scale, kernel=sigmoid;, score=-1605.107 total time=   0.0s
[CV 3/5] END C=10, gamma=scale, kernel=sigmoid;, score=-1742.839 total time=   0.0s
[CV 4/5] END C=10, gamma=scale, kernel=sigmoid;, score=-2419.835 total time=   0.0s
[CV 5/5] END C=10, gamma=scale, kernel=sigmoid;, score=-1632.847 total time=   0.0s
[CV 1/5] END C=10, gamma=auto, kernel=linear;, score=-1973.433 total time=   0.0s
[CV 2/5] END C=10, gamma=auto, kernel=linear;, score=-349.523 total time=   0.0s
[CV 3/5] END C=10, gamma=auto, kernel=linear;, score=-384.747 total time=   0.0s
[CV 4/5] END

[CV 4/5] END .C=100, gamma=0.001, kernel=rbf;, score=-533.156 total time=   0.0s
[CV 5/5] END .C=100, gamma=0.001, kernel=rbf;, score=-414.761 total time=   0.0s
[CV 1/5] END C=100, gamma=0.001, kernel=sigmoid;, score=-2371.544 total time=   0.0s
[CV 2/5] END C=100, gamma=0.001, kernel=sigmoid;, score=-631.647 total time=   0.0s
[CV 3/5] END C=100, gamma=0.001, kernel=sigmoid;, score=-1153.614 total time=   0.0s
[CV 4/5] END C=100, gamma=0.001, kernel=sigmoid;, score=-1458.838 total time=   0.0s
[CV 5/5] END C=100, gamma=0.001, kernel=sigmoid;, score=-954.325 total time=   0.0s
[CV 1/5] END C=100, gamma=0.0001, kernel=linear;, score=-1973.438 total time=   0.0s
[CV 2/5] END C=100, gamma=0.0001, kernel=linear;, score=-349.111 total time=   0.0s
[CV 3/5] END C=100, gamma=0.0001, kernel=linear;, score=-384.748 total time=   0.0s
[CV 4/5] END C=100, gamma=0.0001, kernel=linear;, score=-560.545 total time=   0.0s
[CV 5/5] END C=100, gamma=0.0001, kernel=linear;, score=-578.056 total time=  

[CV 1/5] END C=1000, gamma=0.1, kernel=linear;, score=-1973.436 total time=   0.2s
[CV 2/5] END C=1000, gamma=0.1, kernel=linear;, score=-351.317 total time=   0.1s
[CV 3/5] END C=1000, gamma=0.1, kernel=linear;, score=-384.747 total time=   0.0s
[CV 4/5] END C=1000, gamma=0.1, kernel=linear;, score=-560.583 total time=   0.1s
[CV 5/5] END C=1000, gamma=0.1, kernel=linear;, score=-577.995 total time=   0.1s
[CV 1/5] END C=1000, gamma=0.1, kernel=poly;, score=-2725.621 total time= 1.5min
[CV 2/5] END .C=1000, gamma=0.1, kernel=poly;, score=-448.282 total time=  45.3s
[CV 3/5] END .C=1000, gamma=0.1, kernel=poly;, score=-564.060 total time= 1.2min
[CV 4/5] END .C=1000, gamma=0.1, kernel=poly;, score=-590.290 total time= 2.0min
[CV 5/5] END .C=1000, gamma=0.1, kernel=poly;, score=-524.489 total time=  58.6s
[CV 1/5] END .C=1000, gamma=0.1, kernel=rbf;, score=-1509.247 total time=   0.0s
[CV 2/5] END ..C=1000, gamma=0.1, kernel=rbf;, score=-505.425 total time=   0.0s
[CV 3/5] END ..C=1000,

[CV 1/5] END C=1000, gamma=scale, kernel=linear;, score=-1973.436 total time=   0.2s
[CV 2/5] END C=1000, gamma=scale, kernel=linear;, score=-351.317 total time=   0.1s
[CV 3/5] END C=1000, gamma=scale, kernel=linear;, score=-384.747 total time=   0.0s
[CV 4/5] END C=1000, gamma=scale, kernel=linear;, score=-560.583 total time=   0.1s
[CV 5/5] END C=1000, gamma=scale, kernel=linear;, score=-577.995 total time=   0.1s
[CV 1/5] END C=1000, gamma=scale, kernel=poly;, score=-1943.124 total time=   0.1s
[CV 2/5] END C=1000, gamma=scale, kernel=poly;, score=-371.254 total time=   0.0s
[CV 3/5] END C=1000, gamma=scale, kernel=poly;, score=-356.763 total time=   0.1s
[CV 4/5] END C=1000, gamma=scale, kernel=poly;, score=-521.403 total time=   0.1s
[CV 5/5] END C=1000, gamma=scale, kernel=poly;, score=-538.005 total time=   0.0s
[CV 1/5] END C=1000, gamma=scale, kernel=rbf;, score=-1787.025 total time=   0.0s
[CV 2/5] END C=1000, gamma=scale, kernel=rbf;, score=-324.072 total time=   0.0s
[CV 3

[CV 1/5] END C=10000, gamma=0.001, kernel=linear;, score=-1973.422 total time=   2.3s
[CV 2/5] END C=10000, gamma=0.001, kernel=linear;, score=-351.312 total time=   1.6s
[CV 3/5] END C=10000, gamma=0.001, kernel=linear;, score=-384.748 total time=   1.4s
[CV 4/5] END C=10000, gamma=0.001, kernel=linear;, score=-560.586 total time=   2.0s
[CV 5/5] END C=10000, gamma=0.001, kernel=linear;, score=-578.029 total time=   2.3s
[CV 1/5] END C=10000, gamma=0.001, kernel=poly;, score=-1813.889 total time=   0.0s
[CV 2/5] END C=10000, gamma=0.001, kernel=poly;, score=-372.489 total time=   0.0s
[CV 3/5] END C=10000, gamma=0.001, kernel=poly;, score=-352.739 total time=   0.0s
[CV 4/5] END C=10000, gamma=0.001, kernel=poly;, score=-502.341 total time=   0.0s
[CV 5/5] END C=10000, gamma=0.001, kernel=poly;, score=-426.711 total time=   0.0s
[CV 1/5] END C=10000, gamma=0.001, kernel=rbf;, score=-1945.340 total time=   0.0s
[CV 2/5] END C=10000, gamma=0.001, kernel=rbf;, score=-305.242 total time= 

In [38]:
"""
Best parameters found:  {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Lowest RMSE found:  23.993664181112702
"""


from sklearn.svm import SVR
# best_params_ = {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
svr = SVR(**grid.best_params_)
# svm = SVR(**best_params_)
svr.fit(x_train, y_train)
svr_prediction = svr.predict(x_test)
mse(y_test , svr_prediction)

In [44]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [45]:
x_predict = pd.DataFrame(pd.date_range(date.today(), (date.today() + relativedelta(months=1)),freq='d'), columns=['Date'])
x_predict['day'] = x_predict['Date'].dt.day
x_predict['weekday'] = x_predict['Date'].dt.weekday
x_predict['month'] = x_predict['Date'].dt.month
x_predict = x_predict.set_index('Date')

In [46]:
predictions = x_predict.copy()
predictions['b1_usage_predicted_xgb'] = b1_xgb_model.predict(x_predict)
predictions['b2_usage_predicted_xgb'] = b2_xgb_model.predict(x_predict)
predictions['total_usage_predicted_xgb'] = predictions['b1_usage_predicted_xgb'] + predictions['b2_usage_predicted_xgb']

In [42]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
             'max_features': [ 1, 'sqrt'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10],
             'n_estimators': [100 ,200,300 , 400, 600, ]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

In [ ]:
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

estimators = [
    ('XGB', xgb.XGBRegressor()),
    ('svr', SVR()),
    ('forest', RandomForestRegressor()),
    ('LR' ,Ridge(alpha=1.0))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)
grid_param = {
              "XGB__learning_rate": [0.01, 0.001, 0.1],
              "XGB__n_estimators": [100, 150, 200 , 250 , 300 ],
              "XGB__alpha": [0.1,0.75 , 0.5, 1],
              "XGB__max_depth": [2, 3, 4 , 6, 9 ],
              
              'svr__C': [0.1, 1, 10, 100 ], 
              'svr__gamma': [1, 0.1, 0.01, 0.001 ,'scale', 'auto'],
              'svr__kernel': ['linear', 'rbf', 'sigmoid'],

             'forest__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
             'forest__n_estimators': [100 ,200,300 , 400, 600, ],
             
             'LR__alpha': [0.1 , 0.5 , 1.0 , 1.5]
             }
grid = GridSearchCV(reg, grid_param, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

with open('model.pkl', 'wb') as saved_model:
    pickle.dump(grid, saved_model)

# reg.fit(x_train, y_train)
# reg_prediction = reg.predict(x_test)
# mse(y_test , reg_prediction)

Fitting 5 folds for each of 4752000 candidates, totalling 23760000 fits
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=1, svr__kernel=linear;, score=-1837.528 total time=   1.2s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=1, svr__kernel=linear;, score=-337.057 total time=   1.5s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=1, svr__kernel=linear;, score=-529.273 total time=   1.2s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=1, svr__kernel=linear;, score

[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=0.01, svr__kernel=linear;, score=-303.965 total time=   1.3s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=0.01, svr__kernel=rbf;, score=-1332.606 total time=   1.3s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=0.01, svr__kernel=rbf;, score=-377.364 total time=   1.2s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=0.01, svr__kernel=rbf;, score=-529.262 total time=   1.2s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0

[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=scale, svr__kernel=rbf;, score=-428.481 total time=   1.4s
[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=scale, svr__kernel=rbf;, score=-216.797 total time=   1.2s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=scale, svr__kernel=sigmoid;, score=-1233.623 total time=   1.3s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=0.1, svr__gamma=scale, svr__kernel=sigmoid;, score=-311.755 total time=   1.2s
[CV 3/5] END LR__alpha=0.1, XGB

[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=1, svr__kernel=sigmoid;, score=-430.542 total time=   1.2s
[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=1, svr__kernel=sigmoid;, score=-597.879 total time=   1.2s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.1, svr__kernel=linear;, score=-1878.016 total time=   1.1s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.1, svr__kernel=linear;, score=-378.947 total time=   1.2s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, X

[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.001, svr__kernel=linear;, score=-729.040 total time=   1.1s
[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.001, svr__kernel=linear;, score=-352.313 total time=   1.2s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.001, svr__kernel=rbf;, score=-1435.790 total time=   1.2s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=100, svr__C=1, svr__gamma=0.001, svr__kernel=rbf;, score=-303.990 total time=   1.1s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.

...

[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.1, svr__kernel=rbf;, score=-1876.505 total time=   3.6s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.1, svr__kernel=rbf;, score=-330.529 total time=   3.7s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.1, svr__kernel=rbf;, score=-484.336 total time=   3.7s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.1, svr__kernel=rbf;, score=-840.976 total time=   3.7s
[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__le

[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.001, svr__kernel=sigmoid;, score=-1902.177 total time=   4.1s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.001, svr__kernel=sigmoid;, score=-303.053 total time=   3.7s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.001, svr__kernel=sigmoid;, score=-493.660 total time=   3.9s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=0.001, svr__kernel=sigmoid;, score=-432.391 total time=   3.8s
[CV 5/5] END LR__alpha=0.1,

[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=10, svr__gamma=auto, svr__kernel=sigmoid;, score=-378.371 total time=   3.8s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=1, svr__kernel=linear;, score=-1679.106 total time=   4.1s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=1, svr__kernel=linear;, score=-332.756 total time=   3.9s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=1, svr__kernel=linear;, score=-560.735 total time=   4.2s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0

[CV 5/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=0.01, svr__kernel=linear;, score=-334.209 total time=   4.0s
[CV 1/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=0.01, svr__kernel=rbf;, score=-1621.273 total time=   3.8s
[CV 2/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=0.01, svr__kernel=rbf;, score=-297.263 total time=   3.8s
[CV 3/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=0.01, svr__kernel=rbf;, score=-483.757 total time=   3.6s
[CV 4/5] END LR__alpha=0.1, XGB__alpha=0

[CV 4/5] END LR__alpha=0.1, XGB__alpha=0.1, XGB__learning_rate=0.01, XGB__max_depth=2, XGB__n_estimators=100, forest__max_depth=10, forest__n_estimators=400, svr__C=100, svr__gamma=scale, svr__kernel=rbf;, score=-1034.496 total time=   3.8s


In [47]:
# predictions['total_usage_predicted_xgb'] = xgb_model.predict(x_predict)
predictions

,day,weekday,month,b1_usage_predicted_xgb,b2_usage_predicted_xgb,total_usage_predicted_xgb
Date,,,,,,
2023-04-17,17,0,4,33.658539,21.902981,55.561520
2023-04-18,18,1,4,33.658539,21.902981,55.561520
2023-04-19,19,2,4,33.658539,21.902981,55.561520
2023-04-20,20,3,4,33.658539,21.902981,55.561520
2023-04-21,21,4,4,41.457119,21.902981,63.360100
2023-04-22,22,5,4,41.457119,21.697838,63.154957
2023-04-23,23,6,4,41.457119,22.978781,64.435898
2023-04-24,24,0,4,41.457119,21.697838,63.154957
2023-04-25,25,1,4,41.457119,21.697838,63.154957
